In [ ]:
%use coroutines

import pl.mareklangiewicz.kommand.*
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS
import pl.mareklangiewicz.kommand.core.*
import pl.mareklangiewicz.kommand.core.LsOpt.*
import pl.mareklangiewicz.kommand.gnome.*
import pl.mareklangiewicz.kommand.gnome.GnomeExt.Cmd.list

// TODO NOW: go through this whole notebook again (commented versions too) after refactor; also test input/output sequences (like slow find output)
// TODO NOW: analyze ProcessBuilder.startPipeline and understand assumptions/invariants and make sure my "pipelines" with line sequences are correct.
//    also reproduce example from ProcessBuilder.startPipeline javadoc using my stuff

In [ ]:
    /**
     * Kinda like .exec, but less strict/explicit, because here in notebook we are in more local "experimental" context.
     * 
     * WARNING: Current impl first wait for process to read whole input (blocking) and then starts to consume output.
     * If it deadlocks, that is why.. See CliPlatform.execonsume - same problem
     */
@OptIn(DelicateKommandApi::class)
fun Kommand.x(
    platform: CliPlatform = SYS,
    dir: String? = null,
    vararg useNamedArgs: Unit,
    inContent: String? = null,
    inLines: Sequence<String>? = inContent?.lineSequence(),
    inFile: String? = null,
    outFile: String? = null,
    outFileAppend: Boolean = false,
    errToOut: Boolean = false,
    errFile: String? = null,
    errFileAppend: Boolean = false,
    expectedExit: Int? = 0,
    expectedErr: ((List<String>) -> Boolean)? = { it.isEmpty() },
    outLinesConsumer: ((outLines: Sequence<String>) -> Unit)? = null,
): List<String> {
    require(platform.isRedirectFileSupported || (inFile == null && outFile == null)) { "redirect file not supported here" }
    require(inLines == null || inFile == null) { "Either inLines or inFile or none, but not both" }
    require(outLinesConsumer == null || outFile == null) { "Either outLinesConsumer or outFile or none, but not both" }
    val eprocess = platform.start(this, 
        dir = dir, 
        inFile = inFile, 
        outFile = outFile, 
        outFileAppend = outFileAppend, 
        errToOut = errToOut, 
        errFile = errFile, 
        errFileAppend = errFileAppend
    )
    inLines?.let(eprocess::useInLines)
    outLinesConsumer?.run { eprocess.useOutLines { this(it) } }
    return eprocess
        .waitForResult() // inLines already used
        .unwrap(expectedExit, expectedErr)
}

In [ ]:
SYS.run {
    start(gnometerm(bash(gnomeext(list), pause = true)))
    start(gnometerm(bash(ls { -LongFormat; -All }, pause = true)))
}

In [ ]:
ls { -LongFormat; -All }.x {
    it.take(4).forEach { println("out line: $it") }
}

In [ ]:
LsSamples.lsParentWithSlashes.x().loglns()

In [ ]:
import pl.mareklangiewicz.kommand.konfig.*

SYS.konfigInUserHomeConfigDir().loglns()

In [ ]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.debian.*
import pl.mareklangiewicz.kommand.debian.DpkgAct.*
import pl.mareklangiewicz.kommand.samples.Samples

SYS.run {
    val k =
        Samples.GitHub.help
//        Samples.GitHub.secretListForAbcdK
//        Samples.GitHub.secretSetFakeSecretInAbcdK
//        dpkg { -PrintArch }
//        dpkg(Status("apt"))
//        dpkg(Details("findutils"))
//        sudo(bash("whoami; pwd; ls"))
//        sudo(kommand("aptitude"))
//        sudoEdit("/etc/hosts")
//        sudoEdit("/etc/fstab")
//        kommand("whoami").withSudo(SudoOpt.User(zenityAskForEntryExec("Enter some user name")))
    startInGnomeTermIfUserConfirms(k)
}


In [ ]:
import pl.mareklangiewicz.kommand.debian.*

SYS.dpkgSearchOneCommandExec("find")

In [ ]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.find.*
import kotlin.time.*
import kotlin.time.TimeSource.*

@OptIn(ExperimentalTime::class)
SYS.run {
//    sudoExec(ls("/root/", withHidden = true), inPass = zenityAskForPasswordExec())
//    FindSamples.findAbcIgnoreCase.exec()
//    FindSamples.findSomeSamples.exec()
//    FindSamples.findInKotlinDirNodeModulesDirs.exec().printlns()
//    FindSamples.findInKotlinDirBuildDirs.exec().printlns()
//    FindSamples.findSymLinksToKtsFilesInDepsKt.exec().printlns()
//    FindSamples.findDepthMax2FilesInDepsKtAndRunFileOnEach.exec().printlns()
//    FindSamples.findInKotlinKtFilesModifiedIn24h.exec().printlns()
//    FindSamples.findMyLastWeekKotlinCode.exec().printlns()
    val k = FindSamples.findInKotlinDirNodeModulesDirs
    val p = start(k)
    p.useOutLines {
        it.loglnsWithMillis()
    }
    val r = p.waitForResult()
    println(r)
}

In [ ]:
import pl.mareklangiewicz.kommand.find.*

val llist = SYS.findTypicalDetailsTableExec("..")
llist.map { it.joinToString(",") }.joinToString("\n")


In [ ]:
findMyKotlinCode(
    withGrepRE = "import",
    withModifTime24h = FindExpr.NumArg.LessThan(8)
).checkInIdeap()

In [ ]:
SYS.run {
    startInGnomeTermIfUserConfirms(adb(Adb.Command.pair("192.168.0.125", 38451)))
}